<a href="https://colab.research.google.com/github/lucprosa/dataeng-basic-course/blob/main/spark_streaming/usecase1/script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cloning Repo

https://medium.com/@ashwindesilva/how-to-use-google-colaboratory-to-clone-a-github-repository-e07cf8d3d22b

https://github.com/lucprosa/dataeng-basic-course

In [1]:
from google.colab import drive
drive.mount('/collab/')

Mounted at /collab/


In [2]:
%cd /collab/MyDrive/Collab/repos/

/collab/MyDrive/Collab/repos


In [ ]:
! git clone https://github.com/lucprosa/dataeng-basic-course.git

Cloning into 'dataeng-basic-course'...


In [3]:
! cd /content/
! mkdir /content/files
! mkdir /content/input
! mkdir /content/output
! cp /collab/MyDrive/Collab/repos/dataeng-basic-course/spark_streaming/usecase1/source/* /content/files

/bin/bash: line 1: cd: /content/files: No such file or directory


# Setting up PySpark

In [4]:
%pip install pyspark

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('Test streaming').config('spark.ui.port', '4050').getOrCreate()

# Read CSVs as streaming

In [6]:
from pyspark.sql.types import *
schema = StructType([
StructField('person_ID',IntegerType(),True), StructField('name',StringType(),True), StructField('first',StringType(),True),
StructField('last', StringType(), True),
StructField('middle', StringType(), True),
StructField('email', StringType(), True),
StructField('phone', StringType(), True),
StructField('fax', StringType(), True),
StructField('title', StringType(), True)])

In [58]:
! rm -rf /content/checkpoint

In [7]:
people_df = spark.readStream.format('csv').schema(schema).option('header', True).load('/content/input')

In [11]:
print(people_df.isStreaming)

True


In [12]:
results_df = people_df.groupBy("title").count()

query = (results_df.writeStream
.format('memory')
.queryName('my_query')
.option('checkpointLocation', '/content/checkpoint')
.option('path', '/content/output')
.outputMode('append')
.start()
)

AnalysisException: Append output mode not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark;
Aggregate [title#8], [title#8, count(1) AS count#48L]
+- StreamingRelation DataSource(org.apache.spark.sql.SparkSession@3c069f52,csv,List(),Some(StructType(StructField(person_ID,IntegerType,true),StructField(name,StringType,true),StructField(first,StringType,true),StructField(last,StringType,true),StructField(middle,StringType,true),StructField(email,StringType,true),StructField(phone,StringType,true),StructField(fax,StringType,true),StructField(title,StringType,true))),List(),None,Map(header -> true, path -> /content/input),None), FileSource[/content/input], [person_ID#0, name#1, first#2, last#3, middle#4, email#5, phone#6, fax#7, title#8]


In [10]:
spark.sql("select * from my_query").show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `my_query` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [my_query], [], false


In [57]:
query.stop()